## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 06 – Übungen

### Aufgabe 00.

Überlege dir eine Frage zum Inhalt der Vorlesung, z. B. über einen Punkt, der unklar geblieben ist oder über etwas, worüber du gerne mehr wissen möchtest.  

### Aufgabe 01. Gewichtungen in der Levenshtein-Distanz

In [1]:
pip install music21

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import music21 as m21

Der folgende Code-Block enthält eine Implementierung der Levenshtein-Distanz, wie wir sie in der Vorlesung zur Fugenanalyse genutzt haben.\
Passe diese Implementierung an, indem du dir verschiedene Gewichtungen für die Operationen *Einfügen*, *Löschen* und *Ersetzen* überlegst.\
Welche Gewichtungen könnten für unsere Fugenanalyse sinnvoll sein?

In [3]:
def levenshtein_distance_for_fugues(list1, list2):
    M = len(list1) + 1
    N = len(list2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Listen-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if (i == 0 and j == 0):
                D[i, j] = 0

            elif (i == 0 or j == 0):
                D[i, j] = 0
                
            elif list1[i-1] == list2[j-1]: # Gleichheit
                D[i, j] = D[i-1, j-1]

            elif list1[i-1] != list2[j-1]:   
                D[i,j] = D[i-1,j-1] + 1
            else:
                min_value = min(D[i-1, j], D[i, j-1])
                D[i, j] = min_value + 2

    return D[i, j]

In [4]:
list1 = [1,2,4,5,6,7,8,9,0]
list2 = [0,9,8,7,6,5,4,3,2,1]
print(levenshtein_distance_for_fugues(list1, list2))

9.0


### Aufgabe 02. Computergestützte Fugenanalyse: Subject-Detection mithilfe der Levenshtein-Distanz.

In [5]:
import numpy as np
import music21 as m21

Die folgenden beiden Code-Blöcke enthalten Funktionen aus der Vorlesung.

In [6]:
def notes_to_diatonic_intervals(note_list=[m21.note.Note("C4"), m21.note.Note("E4"), m21.note.Note("G4")]):

    interval_dictionary = {0: 1, 1:2, 2:2, 3:3, 4:3, 5:4, 6:4.5, 7:5, 8:6, 9:6, 10:7, 11:7, 12:8, 13:9, 14:9, 15:10, 16:10, 17:11,
                                    -1:-2, -2:-2, -3:-3, -4:-3, -5:-4, -6:-4.5, -7:-5, -8:-6, -9:-6, -10:-7, -11:-7, -12:-8, -13:-9, -14:-9, -17:-11}

    num_notes = len(note_list)
    interval_list = []

    for i in range(num_notes - 1):

        current_note = note_list[i].pitch.midi
        next_note = note_list[i+1].pitch.midi

        current_interval = next_note - current_note    
        interval_list.append(current_interval)

    diatonic_interval_list = [interval_dictionary[interval] for interval in interval_list]

    return diatonic_interval_list

In [7]:
def notesToDurations(noteList):
    durList = []
    for i in range(len(noteList)-1):
        currentNote = noteList[i].duration.quarterLength
        durList.append(currentNote)
    return durList
        

In [8]:
def subject_finder(subject_intervals, subject_NoteDur, voice_intervals, voice_NoteDur):

    len_subject = len(subject_intervals)

    # Wo finden wir ein Thema?
    subject_indices = []
    # Welche Distanzen zum Original-Thema?
    distances = []
    threshold = 6

    for i in range(len(voice_intervals)):

        current_distance = levenshtein_distance_for_fugues(subject_intervals, voice_intervals[i : i+len_subject])
        current_distance += levenshtein_distance_for_fugues(subject_NoteDur, voice_NoteDur[i : i+len_subject])
        if current_distance <= threshold:
            
            subject_indices.append(i)
            distances.append(float(current_distance))


    return subject_indices, distances

In der Vorlesung haben wir bei der Nutzung der Levenshtein-Distanz zur Erkennung von Fugenthemen nur die Tonhöhen/Intervallen der jeweiligen Stimmen der Fuge berücksichtigt.\
Überlege dir eine Implementierung, die zusätzlich die Notendauern/Rhythmen berücksichtigt. Passe dazu zum Beispiel die Funkion *subject_finder* an.

*Hinweis: Nutze zum Beispiel die Note-Methode .duration.quarterLength*

In [9]:
fugue_score = m21.converter.parse('bach_fugue_cminor.krn')

In [10]:
soprano_notes = list(fugue_score.parts[0].flatten().getElementsByClass(m21.note.Note))
alto_notes = list(fugue_score.parts[1].flatten().getElementsByClass(m21.note.Note))
bass_notes = list(fugue_score.parts[2].flatten().getElementsByClass(m21.note.Note))

soprano_diatonic_intervals = notes_to_diatonic_intervals(soprano_notes)
alto_diatonic_intervals = notes_to_diatonic_intervals(alto_notes)
bass_diatonic_intervals = notes_to_diatonic_intervals(bass_notes)

sopranoDur = notesToDurations(soprano_notes)
altoDur = notesToDurations(alto_notes)
bass_notes = notesToDurations(bass_notes)

In [11]:
subject_diatonic_intervals = alto_diatonic_intervals[:19]
subjectDur = altoDur[:19]

In [12]:
indices, distances = subject_finder(subject_diatonic_intervals, subjectDur, alto_diatonic_intervals, altoDur)

In [13]:
dictFuge = {}
for i in range(len(indices)):
    dictFuge[indices[i]] = distances[i]
print(dictFuge)

{0: 0.0, 120: 2.0, 244: 5.0, 245: 4.0, 246: 2.0, 247: 1.0, 248: 1.0}


### Aufgabe 03. Von der Levenshtein-Distanz zur Levenshtein-Similarity.

In [14]:
import numpy as np
import music21 as m21

Versuche, unsere Implementierung der Levenshtein-Distanz so anzupassen, dass aus ihr ein Ähnlichkeits-Maß (Similarity Measure) wird.\\
Das heißt: dort wo die Levenshtein-Distanz hohe Werte liefert (also Werte, die einer hohen *Unähnlichkeit* entsprechen), soll die Levenshtein-Similarity niedrige Werte liefern -- und umgekehrt.\
Nutze die Levenshtein-Similarity zur Erkennung von Fugenthemen, indem du die Implementierung der Funktion *subject_finder* anpasst.

*Hinweis:* Wähle unter anderem einen geeigneten Threshold-Wert.

In [15]:
def levenshtein_similarity_for_fugues(list1, list2):
    M = len(list1) + 1
    N = len(list2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Listen-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if (i == 0 and j == 0):
                D[i, j] = 0

            elif (i == 0 or j == 0):
                D[i, j] = -1000 #scheiß 1000
                
            elif list1[i-1] == list2[j-1]: # Gleichheit
                D[i, j] = D[i-1, j-1] + 1

            elif list1[i-1] != list2[j-1]:   
                D[i,j] = D[i-1,j-1] - 1
            else:
                min_value = max(D[i-1, j], D[i, j-1])
                D[i, j] = min_value - 3

    return D[i, j]

In [20]:
def subject_finder_Similarity(subject_intervals, subject_NoteDur, voice_intervals, voice_NoteDur):

    len_subject = len(subject_intervals)

    # Wo finden wir ein Thema?
    subject_indices = []
    # Welche Distanzen zum Original-Thema?
    distances = []
    threshold = 30

    for i in range(len(voice_intervals)):

        current_distance = levenshtein_similarity_for_fugues(subject_intervals, voice_intervals[i : i+len_subject])
        current_distance += levenshtein_similarity_for_fugues(subject_NoteDur, voice_NoteDur[i : i+len_subject])
        if current_distance > threshold:
            
            subject_indices.append(i)
            distances.append(float(current_distance))


    return subject_indices, distances

In [21]:
list1 = [1,2,4,5,6,7,8,9,0]
list2 = [0,9,8,7,6,5,4,3,2,1]
print(levenshtein_similarity_for_fugues(list1, list2))

-1009.0


In [22]:
indicesSim, distancesSim = subject_finder_Similarity(subject_diatonic_intervals, subjectDur, alto_diatonic_intervals, altoDur)
print(subject_finder_Similarity(subject_diatonic_intervals, subjectDur, alto_diatonic_intervals, altoDur))

([0, 120], [38.0, 34.0])


In [25]:
dictFuge = {}
for i in range(len(indices)-1):
    dictFuge[indicesSim[i]] = distancesSim[i]
print(dictFuge)

IndexError: list index out of range